In [1]:
import functools

In [2]:
def flatten_boxes(boxes):
    list_of_boxes = functools.reduce(lambda m, box: m + list(map(lambda side: f'{box[0]} {side}', list(box[1]))), boxes, [])
    i = 0
    while(i < len(list_of_boxes)):
        string = list_of_boxes[i].split(' ')
        if string[0] == 'B2' and (string[1] == 'L' or string[1] == 'B'):
            list_of_boxes.pop(i)
        elif string[0] == 'A1' and (string[1] == 'R' or string[1] == 'T'):
            list_of_boxes.pop(i)
        else:
            i+=1
    return list_of_boxes

turn = [('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')]
flatten_boxes(turn)

[&#39;A1 L&#39;,
 &#39;A1 B&#39;,
 &#39;A2 L&#39;,
 &#39;A2 T&#39;,
 &#39;A2 R&#39;,
 &#39;A2 B&#39;,
 &#39;B1 L&#39;,
 &#39;B1 T&#39;,
 &#39;B1 R&#39;,
 &#39;B1 B&#39;,
 &#39;B2 T&#39;,
 &#39;B2 R&#39;]

In [26]:
def remove_move(boxes, move):
    move_box, move_side = move.split(' ')
    clone = boxes[:]

    for i, box in enumerate(clone):
        if box[0] == move_box:
            new_sides = box[1].replace(move_side, '')
            clone[i] = (box[0], new_sides) if new_sides != '' else None
            break

    nonone = list(filter(lambda b: not (b is None), clone))

    return nonone

In [5]:
def is_candidate_closed_by(box, move):
    move_box, move_side = move.split(' ')

    opposites = {
        'T': 'B',
        'B': 'T',
        'L': 'R',
        'R': 'L'
    }

    if box[0] == move_box and box[1] == move_side: # closes box
        return True
    if box[0] != move_box and box[1] == opposites[move_side] and move_box[0] == box[0][0]: # closes same column
        return True
    elif box[0] != move_box and box[1] == opposites[move_side] and move_box[1] == box[0][1]: # closes same row
        return True

    return False

In [27]:
def closed_boxes(boxes, move):
    candidates = list(filter(lambda b: len(b[1]) == 1, boxes))
    closed = list(filter(lambda b: is_candidate_closed_by(b, move), candidates))

    return len(closed)

In [35]:
# boxes [('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')] - primera optimizacion, eliminar jugadas repetidas
def minimax(boxes, max_turn, depth, max_score, min_score):
    if len(boxes) == 0 or depth == 0: # segunda optimizacion, cortar en profundidad y evaluar
        return max_score - min_score

    if max_turn:
        max_eval = -1000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            new_max_score = max_score + closed_boxes(boxes, move) # tercera optimizacion, usar numero de cajas cerradas para ordenar movimientos
            rating = minimax(new_moves, not max_turn, depth - 1, new_max_score, min_score)
            max_eval = max(rating, max_eval) # cuarta optimizacion, usar poda alfa-beta
        return max_eval
    else:
        min_eval = 1000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            new_min_score = min_score + closed_boxes(boxes, move)
            rating = minimax(new_moves, not max_turn, depth - 1, max_score, new_min_score)
            min_eval = min(rating, min_eval)
        return min_eval

minimax([('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')], True, 3, 0, 0)
#minimax([('A1', 'LTRB'), ('A2', 'LTRB')], True, 3, 0, 0)


0